# Load the models

In [42]:
# define the environment path 
local_cobrakbase_path = os.path.join('C:', 'Users', 'Andrew Freiburger','Documents','Argonne','cobrakbase')
os.environ["HOME"] = local_cobrakbase_path

# import the models
import cobrakbase
token = 'WE6CHYRDTJSGOHFIDGPE7WYFT6PRPXJL'
kbase_api = cobrakbase.KBaseAPI(token)
print(dir(model1))

['__add__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__enter__', '__eq__', '__exit__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_annotation', '_compartments', '_contexts', '_from_json', '_id', '_populate_solver', '_repr_html_', '_set_id_with_model', '_solver', '_to_json', '_to_object', '_tolerance', '_trimmed', '_trimmed_genes', '_trimmed_reactions', 'add_boundary', 'add_cons_vars', 'add_groups', 'add_metabolites', 'add_reaction', 'add_reactions', 'annotation', 'biomasses', 'boundary', 'compartments', 'constraints', 'copied', 'copy', 'copy_source_inaccessible', 'created', 'creator', 'data', 'data_keys', 'demands', 'description', 'epoch', 'exchanges', 'exclude_dict', 'gapfillings', 'gapgens', '

# Test functionality

## Align exchange reactions

In [96]:
%run ../ModelSEEDpy/modelseedpy/core/mscompatibility.py

model1 = kbase_api.get_from_ws("iML1515",76994)
model2 = kbase_api.get_from_ws("iSB1139.kb.gf",30650)
mscompat = MSCompatibility(modelseed_db_path = os.path.join('..', 'ModelSEEDDatabase'))

# model1
before_model1_rxns = set()
for rxn in model1.reactions:
    before_model1_rxns.add(rxn.reaction)

model1, unknown_model1_ids = mscompat.exchanges(model1)

after_model1_rxns = set()
for rxn in model1.reactions:
    after_model1_rxns.add(rxn.reaction)
print('model1 changes:\n', before_model1_rxns.symmetric_difference(after_model1_rxns))

# model2
before_model2_rxns = set()
for rxn in model2.reactions:
    before_model2_rxns.add(rxn.reaction)

model2, unknown_model2_ids = mscompat.exchanges(model2)

after_model2_rxns = set()
for rxn in model2.reactions:
    after_model2_rxns.add(rxn.reaction)
print('model2 changes:\n', before_model2_rxns.symmetric_difference(after_model2_rxns))


original met_rxn	 cpd00532_e0 <=> cpd00532_p0
new met_rxn		 cpd00099_e0 <=> cpd00532_p0

original met_rxn	 cpd00532_e0 <=> 
new met_rxn		 cpd00099_e0 <=> 

original met_rxn	 cpd15459_e0 <=> 
new met_rxn		 cpd15493_e0 <=> 

original met_rxn	 cpd00001_c0 + cpd00002_c0 + cpd15459_p0 --> cpd00008_c0 + cpd00009_c0 + cpd00067_c0 + cpd15459_e0
new met_rxn		 cpd00001_c0 + cpd00002_c0 + cpd15459_p0 --> cpd00008_c0 + cpd00009_c0 + cpd00067_c0 + cpd15493_e0

original met_rxn	 cpd00637_e0 <=> 
new met_rxn		 cpd00060_e0 <=> 

original met_rxn	 cpd00637_e0 <=> cpd00637_p0
new met_rxn		 cpd00060_e0 <=> cpd00637_p0

original met_rxn	 cpd00035_e0 <=> 
new met_rxn		 cpd00085_e0 <=> 

original met_rxn	 cpd00035_e0 <=> cpd00035_p0
new met_rxn		 cpd00085_e0 <=> cpd00035_p0

original met_rxn	 cpd00084_e0 <=> 
new met_rxn		 cpd00587_e0 <=> 

original met_rxn	 cpd00084_e0 <=> cpd00084_p0
new met_rxn		 cpd00587_e0 <=> cpd00084_p0

original met_rxn	 cpd00159_e0 <=> cpd00159_p0
new met_rxn		 cpd00221_e0 <=> cpd

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:167: UserWarning: The tartr-D-e_e0 metabolite in the TARTRDtex_e0 reaction could not be corrected.
  warn(f'The {met.id} metabolite in the {rxn.id} reaction could not be corrected.')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:167: UserWarning: The tartr-D-c_c0 metabolite in the TARTt2__DASH__3pp_c0 reaction could not be corrected.
  warn(f'The {met.id} metabolite in the {rxn.id} reaction could not be corrected.')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:167: UserWarning: The tartr-D-c_c0 metabolite in the SUCTARTtpp_c0 reaction could not be corrected.
  warn(f'The {met.id} metabolite in the {rxn.id} reaction could not be corrected.')



original ID	 sfp-c_c0 	 6-deoxy-6-sulphofructose 1-phosphate
new ID		 cpd32891 	 6-deoxy-6-sulphofructose 1-phosphate

original ID	 xylu-D-c_c0 	 D-Xylulose
new ID		 cpd00259 	 D-Xylulose

original ID	 metsox-R-L-p_p0 	 L-methionine-R-sulfoxide
new ID		 cpd14720 	 L-methionine-R-sulfoxide

original met_rxn	 cpd11421_c0 + metsox-R-L-c_c0 --> cpd00001_c0 + cpd00060_c0 + cpd11420_c0
new met_rxn		 cpd11421_c0 + cpd14720 --> cpd00001_c0 + cpd00060_c0 + cpd11420_c0

original met_rxn	 cpd00001_c0 + cpd00002_c0 + cpd14720 --> cpd00008_c0 + cpd00009_c0 + cpd00067_c0 + metsox-R-L-c_c0
new met_rxn		 cpd00001_c0 + cpd00002_c0 + cpd14720 --> cpd00008_c0 + cpd00009_c0 + cpd00067_c0 + metsox-R-L-c_c0

original met_rxn	 cpd00025_c0 + cpd00060_c0 --> cpd00001_c0 + metsox-R-L-c_c0
new met_rxn		 cpd00025_c0 + cpd00060_c0 --> cpd00001_c0 + cpd14720

original met_rxn	 metsox-R-L-e_e0 <=> cpd14720
new met_rxn		 metsox-R-L-e_e0 <=> cpd14720

original met_rxn	 metsox-R-L-e_e0 <=> 
new met_rxn		 cpd14720 <=> 

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:167: UserWarning: The metsox-R-L-c_c0 metabolite in the METSOX2abcpp_c0 reaction could not be corrected.
  warn(f'The {met.id} metabolite in the {rxn.id} reaction could not be corrected.')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:167: UserWarning: The metsox-R-L-e_e0 metabolite in the METSOX2tex_e0 reaction could not be corrected.
  warn(f'The {met.id} metabolite in the {rxn.id} reaction could not be corrected.')



original ID	 sf-c_c0 	 6-deoxy-6-sulphofructose
new ID		 cpd35501 	 6-deoxy-6-sulphofructose

original ID	 xylu-L-c_c0 	 L-Xylulose
new ID		 cpd00261 	 L-Xylulose
model1 changes:
 {'3.0 cpd00067_p0 + tartr-D-p_p0 --> 3.0 cpd00067_c0 + tartr-D-c_c0', 'cpd00060_e0 <=> cpd00637_p0', 'cpd00653_e0 <=> cpd00653_p0', 'cpd00637_e0 <=> cpd00637_p0', '2.0 cpd22495 --> 2.0 cpd00067_c0 + cpd15511_p0 + 2.0 cpd25775_c0', 'cpd00067_p0 + cpd00259_p0 --> cpd00067_c0 + xylu-L-c_c0', 'cpd01067_c0 --> cpd00261', 'cpd11421_c0 + cpd14720 --> cpd00001_c0 + cpd00060_c0 + cpd11420_c0', 'cpd19018 <=> ', 'ampnt-c_c0 + cpd00022_c0 --> acampnt-c_c0 + cpd00010_c0', 'cpd14720 <=> ', 'cpd00550_e0 <=> cpd00054_p0', 'sq-c_c0 --> sf-c_c0', 'ampnt-c_c0 + cpd00022_c0 --> cpd00010_c0 + cpd35393', 'cpd00085_e0 <=> cpd00117_p0', 'cpd00637_e0 <=> ', 'cpd00117_e0 <=> ', 'cpd00653_e0 <=> ', 'cpd00035_e0 <=> cpd00035_p0', 'cpd00532_e0 <=> ', 'cpd00159_e0 <=> cpd00159_p0', 'cpd00002_c0 + cpd00259 --> cpd00008_c0 + cpd00067_c0 + 

...


original ID	 r54_c0 	 2-oxoarginine
new ID		 cpd02364 	 2-oxoarginine

original ID	 r1012_c0 	 fructose-1-phosphate
new ID		 cpd00802 	 fructose-1-phosphate

original met_rxn	 cpd00024 --> r1423_e0
new met_rxn		 cpd00024 --> r1423_e0

original met_rxn	 r1423_e0 <=> 
new met_rxn		 cpd00024 <=> 

original met_rxn	 cpd00067_e0 + r1423_e0 --> cpd00024 + cpd00067_c0
new met_rxn		 cpd00067_e0 + r1423_e0 --> cpd00024 + cpd00067_c0

original ID	 r1029_c0 	 GDP-D-rhamnose
new ID		 cpd01999 	 GDP-D-rhamnose

original ID	 r1032_c0 	 GDP-mannuronate
new ID		 cpd00719 	 GDP-mannuronate

original ID	 r1037_c0 	 geranyl-PP
new ID		 cpd23795 	 geranyl-PP

original ID	 r16_c0 	 Tetradecanoyl-CoA (n-C14:0CoA)
new ID		 cpd01695 	 Tetradecanoyl-CoA (n-C14:0CoA)

original met_rxn	 cpd00015_c0 + cpd01695 <=> cpd00982_c0 + r15_c0
new met_rxn		 cpd00015_c0 + cpd01695 <=> cpd00982_c0 + cpd03127

original met_rxn	 cpd03115_c0 <=> cpd00001_c0 + r15_c0
new met_rxn		 cpd03115_c0 <=> cpd00001_c0 + cpd03127

origin

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:167: UserWarning: The r1446_c0 metabolite in the MERXN__DASH__1_c0 reaction could not be corrected.
  warn(f'The {met.id} metabolite in the {rxn.id} reaction could not be corrected.')



original ID	 r1431_c0 	 S
new ID		 cpd00074 	 S

original ID	 r1432_c0 	 S-2-methyl-butyryl-CoA
new ID		 cpd00760 	 S-2-methyl-butyryl-CoA

original met_rxn	 cpd00051_c0 --> r241_b0
new met_rxn		 cpd00051_c0 --> cpd00051

original met_rxn	 cpd00035_c0 --> r240_b0
new met_rxn		 cpd00035_c0 --> cpd00035

original ID	 r1074_c0 	 guanosine 5'-diphosphate,3'-diphosphate
new ID		 cpd00905 	 guanosine 5'-diphosphate,3'-diphosphate

original met_rxn	 cpd00053_c0 --> r247_b0
new met_rxn		 cpd00053_c0 --> cpd00053

original met_rxn	 cpd00119_c0 --> r248_b0
new met_rxn		 cpd00119_c0 --> cpd00119

original met_rxn	 cpd00322_c0 --> r249_b0
new met_rxn		 cpd00322_c0 --> cpd00322

original met_rxn	 cpd00266_c0 --> r243_b0
new met_rxn		 cpd00266_c0 --> cpd00266

original met_rxn	 cpd00084_c0 --> r244_b0
new met_rxn		 cpd00084_c0 --> cpd00084

original met_rxn	 cpd00023_c0 --> r245_b0
new met_rxn		 cpd00023_c0 --> cpd00023

original met_rxn	 cpd00033_c0 --> r230_b0
new met_rxn		 cpd00033_c0 --> cpd000

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:167: UserWarning: The r274_b0 metabolite in the MERXN__DASH__70_c0 reaction could not be corrected.
  warn(f'The {met.id} metabolite in the {rxn.id} reaction could not be corrected.')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:167: UserWarning: The r277_e0 metabolite in the MTRXN__DASH__36_c0 reaction could not be corrected.
  warn(f'The {met.id} metabolite in the {rxn.id} reaction could not be corrected.')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:167: UserWarning: The r277_e0 metabolite in the MIRXN__DASH__1_e0 reaction could not be corrected.
  warn(f'The {met.id} metabolite in the {rxn.id} reaction could not be corrected.')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:167: UserWarning: The r293_e0 metabolite in the MTRXN__DASH__50__DASH__MTRXN_


original ID	 r469_c0 	 2-methyl-3-hydroxybutyryl-CoA
new ID		 cpd02691 	 2-methyl-3-hydroxybutyryl-CoA

original ID	 r453_c0 	 xanthosine-5-phosphate
new ID		 cpd00497 	 xanthosine-5-phosphate

original ID	 r458_c0 	 2-maleylacetate
new ID		 cpd01498 	 2-maleylacetate

original met_rxn	 cpd00137_c0 --> r209_b0
new met_rxn		 cpd00137_c0 --> cpd00137

original met_rxn	 cpd00098_c0 --> r208_b0
new met_rxn		 cpd00098_c0 --> cpd00098

original met_rxn	 cpd00242_c0 --> r207_b0
new met_rxn		 cpd00242_c0 --> cpd00242

original met_rxn	 cpd00041_c0 --> r205_b0
new met_rxn		 cpd00041_c0 --> cpd00041

original met_rxn	 cpd00013_c0 --> r204_b0
new met_rxn		 cpd00013_c0 --> cpd19013

original met_rxn	 cpd01092_c0 --> r203_b0
new met_rxn		 cpd01092_c0 --> cpd01092

original met_rxn	 cpd00004_c0 + cpd00067_c0 + cpd15570_c0 --> cpd00003_c0 + r404_c0
new met_rxn		 cpd00004_c0 + cpd00067_c0 + cpd15570_c0 --> cpd00003_c0 + cpd01695

original met_rxn	 cpd00067_c0 + r404_c0 + r58_c0 --> cpd00011_c0 + cpd1

## Compare the models

In [82]:
%run ../ModelSEEDpy/modelseedpy/core/mscompatibility.py

model1 = kbase_api.get_from_ws("iML1515",76994)
model2 = kbase_api.get_from_ws("iSB1139.kb.gf",30650)
mscompat = MSCompatibility(modelseed_db_path = os.path.join('..', 'ModelSEEDDatabase'))

misaligned_reactions, model1, model2 = mscompat.compare_models(model1, model2)
misaligned_reactions, model1, model2 = mscompat.compare_models(model1, model2, False)



616 of the 1877 model_1 metabolites and 1196 model_2 metabolites are shared and were compared.

misaligned rxn01256_c0 names
  model1   Chorismate pyruvatemutase_c0  model2   AroQ (periplasmic) chorismate mutase precursor

misaligned rxn01275_c0 names
  model1   ATP:D-Gluconate 6-phosphotransferase_c0  model2   Glucokinase

misaligned rxn01740_c0 names
  model1   Shikimate:NADP+ 3-oxidoreductase_c0  model2   shikimate dehydrogenase__shikimate dehydrogenase

misaligned rxn00147_c0 names
  model1   ATP:pyruvate,water phosphotransferase_c0  model2   putative phosphoenolpyruvate synthase

misaligned rxn20661_c0 names
  model1   quinolinate synthase_c0  model2   NadA; quinolinate synthetase A

misaligned rxn02405_c0 names
  model1   CTP:3-deoxy-D-manno-octulosonate cytidylyltransferase_c0  model2   3-deoxy-manno-octulosonate cytidylyltransferase

misaligned rxn04016_c0 names
  model1   S-adenosyl-L-methionine:(E)-prop-1-ene-1,2,3-tricarboxylate 2'-O-methyltransferase_c0  model2   trans-ac

## Standardize the models

In [127]:
%run ../ModelSEEDpy/modelseedpy/core/mscompatibility.py

model1 = kbase_api.get_from_ws("iML1515",76994)
model2 = kbase_api.get_from_ws("iSB1139.kb.gf",30650)
mscompat = MSCompatibility(modelseed_db_path = os.path.join('..', 'ModelSEEDDatabase'))

# model1
before_model1_rxns = set()
for rxn in model1.reactions:
    before_model1_rxns.add(rxn.name)

model1 = mscompat.standardize_MSD(model1)

after_model1_rxns = set()
for rxn in model1.reactions:
    after_model1_rxns.add(rxn.name)
print('model1 change:\n', before_model1_rxns.symmetric_difference(after_model1_rxns))

# model2
before_model2_rxns = set()
for rxn in model2.reactions:
    before_model2_rxns.add(rxn.name)

model2 = mscompat.standardize_MSD(model2)

after_model2_rxns = set()
for rxn in model2.reactions:
    after_model2_rxns.add(rxn.name)
print('model2 change:\n', before_model2_rxns.symmetric_difference(after_model2_rxns))

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:63: UserWarning: The mchtbs6p-c_c0 metabolite is not captured by the ModelSEED Database
  warn(f'The {met.id} metabolite is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:63: UserWarning: The bmoco-c_c0 metabolite is not captured by the ModelSEED Database
  warn(f'The {met.id} metabolite is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:63: UserWarning: The sufbcd-4fe4s-c_c0 metabolite is not captured by the ModelSEED Database
  warn(f'The {met.id} metabolite is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:63: UserWarning: The prcp-c_c0 metabolite is not captured by the ModelSEED Database
  warn(f'The {met.id} metabolite is not captured

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The MMM_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The ASO3tex_e0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The H2tpp_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The GLYt4pp_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The KARA1_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The GLYBt3pp_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The PYDAMtex_e0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The GALtex_e0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The CGLYabcpp_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The CHLt2pp_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The XYLK2_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The PROt2rpp_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Databa

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The CU1abcpp_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The MALDtex_e0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The ALLTNtex_e0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The Ftpp_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database

...

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The MIRXN__DASH__19_e0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The ENZRXN3EM__DASH__5336_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The ENZRXN3EM__DASH__1128_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The MVRXN__DASH__172__DASH__MVRXN__DASH__178_c0 reaction is not captured by the ModelSEED Database
  

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The ENZRXN3EM__DASH__586_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The ENZRXN3EM__DASH__1547_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The ENZRXN3EM__DASH__5326__DASH__ENZRXN3EM__DASH__5328_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The ENZRXN3EM__DASH__4729__DASH__MTRXN__DASH__2_c0 reaction is not 

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The ENZRXN3EM__DASH__1535_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The ENZRXN3EM__DASH__1218_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The ENZRXN3EM__DASH__912_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The ENZRXN3EM__DASH__1478_c0 reaction is not captured by the ModelSEED Database
  warn(f'The {rx

model2 change:
 {'putative aspartate racemase', 'glyceraldehyde 3-phosphate dehydrogenase _glyceraldehyde-3-phosphate dehydrogenase', 'Phosphoenolpyruvate:UDP-N-acetyl-D-glucosamine 1-carboxyvinyl-transferase', '3-oxoacyl-[acyl-carrier-protein] reductase (n-C8:0)', '2-octaprenyl-6-methoxy-1,4-benzoquinone methylase', 'precorrin-2 dehydrogenase__precorrin-2 dehydrogenase', 'Succinyl-CoA:L-homoserine O-succinyltransferase', 'FMN adenylyltransferase', 'succinate dehydrogenase', 'RXN0-5473.c', 'acyl-CoA dehydrogenase (butanoyl-CoA)__acyl-CoA dehydrogenase (butanoyl-CoA)__butyryl-CoA dehydrogenase', '(R)-S-Lactoylglutathione hydrolase', 'putative phosphoenolpyruvate synthase', 'UDP-N-acetylmuramate:NADP+ oxidoreductase', 'bifunctional PutA protein', 'O3-acetyl-L-serine:hydrogen-sulfide 2-amino-2-carboxyethyltransferase', 'PydA; dihydropyrimidine dehydrogenase', 'adenosylhomocysteinase', 'L-Aspartate-4-semialdehyde:NADP+ oxidoreductase (phosphorylating)', 'N2-succinyl-L-arginine iminohydrola

C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The EX_cpd00242_e0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The EX_cpd00703_e0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The EX_r1400_e0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the ModelSEED Database')
C:\Users\Andrew Freiburger\Documents\Argonne\ModelSEEDpy\modelseedpy\core\mscompatibility.py:82: UserWarning: The EX_cpd11949_e0 reaction is not captured by the ModelSEED Database
  warn(f'The {rxn.id} reaction is not captured by the Mode

# Brainstorming

In [120]:
for rxn in model1.reactions:
    print(dir(rxn))
    for pro in rxn.products:
        print(pro)
    for met in rxn.metabolites:
        print(dir(met))
        print(met.elements)
        break
    break

['__add__', '__class__', '__copy__', '__deepcopy__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__isub__', '__le__', '__lt__', '__module__', '__mul__', '__ne__', '__new__', '__radd__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__sub__', '__subclasshook__', '__weakref__', '_annotation', '_associate_gene', '_check_bounds', '_dissociate_gene', '_gene_reaction_rule', '_genes', '_get_rev_and_max_min_flux', '_id', '_lower_bound', '_metabolites', '_model', '_repr_html_', '_set_id_with_model', '_to_json', '_update_awareness', '_upper_bound', 'add_metabolites', 'aliases', 'annotation', 'boundary', 'bounds', 'build_reaction_from_string', 'build_reaction_string', 'check_mass_balance', 'compartment', 'compartments', 'copy', 'dblinks', 'delete', 'flux', 'flux_expression', 'forward_variabl

In [101]:
xs = {'a': 1, 'b': 2, 'c':3}
xs.update({'a':4})
print(xs)

a = b = c = d = None
print(a,b,c,d)

{'a': 4, 'b': 2, 'c': 3}
None None None None


In [21]:
from numpy import negative
import re

# parse the reaction string
reaction = "(1) cpd00002[0] + (1) cpd00222[0] => (1) cpd00008[0] + (1) cpd00067[0] + (1) cpd00284[0]"
compounds = reaction.split('=>')
reactant, product = compounds[0], compounds[1]
reactants = [x.strip() for x in reactant.split('+')]
products = [x.strip() for x in product.split('+')]
reactant_met = [x.split(' ') for x in reactants]
product_met = [x.split(' ') for x in products]

# assemble a dictionary for the reaction
reaction_dict = {}
for met in reactant_met:
    stoich = float(re.search('(\d+)', met[0]).group())
    met[1] = met[1].replace('[0]', '_c0')
    met[1] = met[1].replace('[1]', '_c0')
    reaction_dict[met[1]] = negative(stoich)
for met in product_met:
    stoich = float(re.search('(\d+)', met[0]).group())
    met[1] = met[1].replace('[0]', '_c0')
    met[1] = met[1].replace('[1]', '_c0')
    reaction_dict[met[1]] = stoich
    

print(reactant_met, product_met)
print(reaction_dict)

[['(1)', 'cpd00002_c0'], ['(1)', 'cpd00222_c0']] [['(1)', 'cpd00008_c0'], ['(1)', 'cpd00067_c0'], ['(1)', 'cpd00284_c0']]
{'cpd00002_c0': -1.0, 'cpd00222_c0': -1.0, 'cpd00008_c0': 1.0, 'cpd00067_c0': 1.0, 'cpd00284_c0': 1.0}


In [20]:
for met in reactant_met:
    print(met)
    met_id = met[1].replace('[0]', '_c0')
    print(met_id)

['(1)', 'cpd00002[0]']
cpd00002_c0
['(1)', 'cpd00222[0]']
cpd00222_c0
